In [1]:
import csv, random, numpy as np
from keras.models import load_model, Sequential,model_from_json
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Cropping2D
from keras.preprocessing.image import img_to_array, load_img, flip_axis, random_shift
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2, activity_l2

from keras.callbacks import Callback,ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from math import floor

import cv2
from PIL import Image
import json



Using TensorFlow backend.


In [2]:
def model(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

#    conv_layers1, conv_layers2, dense_layers = [24, 36, 48], [64, 64], [1024, 512]
    dense_layers = [1024, 512]
    
    model = Sequential()
    
    model.add(Cropping2D(cropping=((40,20), (0,0)), input_shape=shape))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))

    model.add(Convolution2D(32,5, 5,  border_mode = 'valid', subsample=(1, 1),W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
#    model.add(Dropout(0.50))
     
    model.add(Convolution2D(32,5, 5,  border_mode = 'valid', subsample=(1, 1), W_regularizer=l2(0.001), activation='elu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
   

    model.add(Convolution2D(64,3, 3, border_mode = 'valid', subsample=(1, 1), W_regularizer=l2(0.001), activation='elu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))

    model.add(Convolution2D(128, 3, 3,  border_mode = 'valid', subsample=(1, 1), W_regularizer=l2(0.001), activation='elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.50))
    
#    model.add(Convolution2D(128, 3, 3, subsample=(1, 1), activation='elu'))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
    
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl, activation='elu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model

In [3]:
def get_X_y_test(data_file, flag='unix', turn='left'):
    """Read the log file and turn it into X/y pairs. Add an offset to left images, remove from right images."""
    X, y = [], []
    right, left, base, bridge, start_left, start_right = [], [], [], [],[], []
    #    images = []
    steering_offset = 0.35
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):

            if(flag == 'window' and turn == 'right'):
                print('window right file loaded')
                if (float(speed) > 9 and float(steering_angle) > 0.04):
 #               if (float(speed) > 0 and float(steering_angle) > 0.04):
                    print("window right steering angle: %s" % steering_angle )
                
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                    
                    X += [left_img.strip()]
                    y += [float(steering_angle) + steering_offset]
                    right.append(float(steering_angle) + steering_offset)
                    
                    X += [right_img.strip()]
                    y += [float(steering_angle) - steering_offset]
                    
                    right.append(float(steering_angle) - steering_offset)

                    X += [center_img.strip()]
                    y += [float(steering_angle)]
                    
                    right.append(float(steering_angle))
                            #                        shuffle(X, y)
                    shuffle(X, y)

            elif(flag == 'window' and turn == 'left'):
                print('window left file loaded')
                if (float(speed) > 5 and float(steering_angle) < -0.04):
#                if (float(speed) > 10 and float(steering_angle) < -0.04):
#                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    print("window left steering angle: %s" % steering_angle )
          
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                        
                    X += [left_img.strip()]
                    y += [float(steering_angle) + steering_offset]
                    
                    left.append(float(steering_angle) + steering_offset)
                    

                    X += [right_img.strip()]
                    y += [float(steering_angle) - steering_offset]
                    
                    left.append(float(steering_angle) - steering_offset)
                    

                    X += [center_img.strip()]
                    y += [float(steering_angle)]
                    
                    left.append(float(steering_angle))
                # shuffle(X, y)

                    shuffle(X, y)
                    
            elif(flag == 'window' and turn == 'bridge'):
                print('window brige file loaded')
                if (float(speed) > 20):
#                if (float(speed) > 10 and float(steering_angle) < -0.04):
#                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    print("window brige file steering angle: %s" % steering_angle )
          
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                        
                    X += [left_img.strip()]
                    y += [float(steering_angle) + steering_offset]
                    
                    bridge.append(float(steering_angle) + steering_offset)
                    

                    X += [right_img.strip()]
                    y += [float(steering_angle) - steering_offset]
                    
                    bridge.append(float(steering_angle) - steering_offset)
                    

                    X += [center_img.strip()]
                    y += [float(steering_angle)]
                    
                    bridge.append(float(steering_angle))
                # shuffle(X, y)

                    shuffle(X, y)
                    
            elif(flag == 'window' and turn == 'start_left'):
                print('window start_left file loaded')
                if ( float(steering_angle) < -0.04 and float(speed) > 5):
#                if (float(speed) > 10 and float(steering_angle) < -0.04):
#                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    print("window brige file steering angle: %s" % steering_angle )
          
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                        
                    X += [left_img.strip()]
                    y += [float(steering_angle) + steering_offset]
                    
                    start_left.append(float(steering_angle) + steering_offset)
                    

                    X += [right_img.strip()]
                    y += [float(steering_angle) - steering_offset]
                    
                    start_left.append(float(steering_angle) - steering_offset)
                    

                    X += [center_img.strip()]
                    y += [float(steering_angle)]
                    
                    start_left.append(float(steering_angle))
                # shuffle(X, y)

                    shuffle(X, y)
                    
            elif(flag == 'window' and turn == 'start_right'):
                print('window start_right file loaded')
                if ( float(steering_angle) > 0.04 and float(speed) > 5):
#                if (float(speed) > 10 and float(steering_angle) < -0.04):
#                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    print("window brige file steering angle: %s" % steering_angle )
          
                    left_img = left_img.split('\\')[-1]
                    right_img = right_img.split('\\')[-1]
                    center_img = center_img.split('\\')[-1]
                        
                    X += [left_img.strip()]
                    y += [float(steering_angle) + steering_offset]
                    
                    start_right.append(float(steering_angle) + steering_offset)
                    

                    X += [right_img.strip()]
                    y += [float(steering_angle) - steering_offset]
                    
                    start_right.append(float(steering_angle) - steering_offset)
                    

                    X += [center_img.strip()]
                    y += [float(steering_angle)]
                    
                    start_right.append(float(steering_angle))
                # shuffle(X, y)

                    shuffle(X, y)

            elif(flag == 'linux'):
 #               print('linux file loaded')

                    if (float(steering_angle) > 0.35 or float(steering_angle) < -0.35):
                     
                        left_img = left_img.split('/')[-1]
                        right_img = right_img.split('/')[-1]
                        center_img = center_img.split('/')[-1]
        
                        X += [left_img.strip()]
                        y += [float(steering_angle) + steering_offset]
                
                        base.append(float(steering_angle) + steering_offset)

                        X += [right_img.strip()]
                        y += [float(steering_angle) - steering_offset]
                        
                        base.append(float(steering_angle) - steering_offset)
                       

                        X += [center_img.strip()]
                        y += [float(steering_angle)]
                        
                        base.append(float(steering_angle))

                        shuffle(X, y)
                        
            elif(flag == 'window' and turn == 'recovery'):
                print("window")
    
 #   print("riht anglie sum %s " % sum(right))    
 #   print("left anglie sum %s " % sum(left)) 
    num1 = sum(right)
    num2 = sum(left)
    print("additional data, right turn steering anglie sum :")
    print(sum(right))
    print("additional data, left turn steering anglie sum :")
    print(sum(left))
    print("udacity data anglie sum :")
    print(sum(base))
    print("additional data, bridge sum ::")
    print(sum(bridge))
    print("additional data, start_left sum ::")
    print(sum(start_left))
    print("additional data, start_right sum ::")
    print(sum(start_right))
    return X, y

In [4]:
def get_X_y(data_file, flag='unix', turn='left'):
  
    X, y = [], []
    base = []
#    images = []
    steering_offset = 0.35
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):
            
              if (float(steering_angle) > 0.35 or float(steering_angle) < -0.35):
                     
       
                        left_img = left_img.split('/')[-1]
                        right_img = right_img.split('/')[-1]
                        center_img = center_img.split('/')[-1]
        
                        X += [left_img.strip()]
                        y += [float(steering_angle) + steering_offset]
                
                        base.append(float(steering_angle) + steering_offset)

                        X += [right_img.strip()]
                        y += [float(steering_angle) - steering_offset]
                        
                        base.append(float(steering_angle) - steering_offset)
                       

                        X += [center_img.strip()]
                        y += [float(steering_angle)]
                        
                        base.append(float(steering_angle))

                        shuffle(X, y)

    return X, y

In [5]:
data_dir2 = ('../20170329_1/driving_log.csv','window','left')
data_dir3 = ('../20170329_2/driving_log.csv','window','right')
data_dir4 = ('../20170329_3/driving_log.csv','window','bridge')
data_dir5 = ('../20170329_4/driving_log.csv','window','start_left')
data_dir6 = ('../20170329_5/driving_log.csv','window','start_right')
data_dir7 = ('../20170329_6/driving_log.csv','window','left')

In [6]:
def merge_data6(dir1, dir2, dir3, dir4, dir5, dir6):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y_test(dir2[0], flag=dir2[1], turn=dir2[2])
    
    X3, y3 = get_X_y_test(dir3[0], flag=dir3[1],turn=dir3[2])
    
    X4, y4 = get_X_y_test(dir4[0], flag=dir4[1],turn=dir4[2])
    
    X5, y5 = get_X_y_test(dir5[0], flag=dir5[1],turn=dir5[2])
    
    X6, y6 = get_X_y_test(dir6[0], flag=dir6[1],turn=dir6[2])
    
   
    X = X2 + X1 + X3+ X4 + X5 + X6
    y = y2 + y1 + y3 +y4  + y5 + y6
    

    return X,y

In [7]:
def merge_data4(dir1, dir2, dir3, dir4):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y_test(dir2[0], flag=dir2[1], turn=dir2[2])
    
    X3, y3 = get_X_y_test(dir3[0], flag=dir3[1],turn=dir3[2])
    
    X4, y4 = get_X_y_test(dir4[0], flag=dir4[1],turn=dir4[2])
    
   
    X = X2 + X1 + X3+ X4
    y = y2 + y1 + y3 +y4  
    

    return X,y

In [8]:
def merge_data3(dir1, dir2, dir3):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y_test(dir2[0], flag=dir2[1], turn=dir2[2])
    
    X3, y3 = get_X_y_test(dir3[0], flag=dir3[1], turn=dir3[2])
    
   
    X = X2 + X1 + X3
    y = y2 + y1 + y3 
    

    return X,y

In [9]:
def merge_data1(dir1):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1], turn=dir1[2])
    
   
    X = X1
    y =  y1 
    

    return X,y

def merge_data1_2(dir1):

    X1, y1 = get_X_y_test(dir1[0], flag=dir1[1], turn=dir1[2])
    
   
    X = X1
    y =  y1 
    

    return X,y



In [10]:
def random_darken(image):
    """Given an image (from Image.open), randomly darken a part of it."""
    w, h = image.size

    # Make a random box.
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(x1, w), random.randint(y1, h)

    # Loop through every pixel of our box (*GASP*) and darken.
    for i in range(x1, x2):
        for j in range(y1, y2):
            new_value = tuple([int(x * 0.5) for x in image.getpixel((i, j))])
            image.putpixel((i, j), new_value)
    return image

In [11]:
def split_data(data, label, split = 0.3):
    
    train_samples, validation_samples, train_labels, validation_labels = train_test_split(
    data,
    label,
    test_size=split,
    random_state=832289)
    
    return train_samples, validation_samples, train_labels, validation_labels

In [12]:
def process_image_test(image, steering_angle, augment=True, shape=(160,320)):
    """Process and augment an image."""

    pil_image = Image.fromarray(image)

    if augment and random.random() < 0.5:
        pil_image = random_darken(pil_image)  # before numpy'd

    image = img_to_array(pil_image)
        
    if augment:
        image = random_shift(image, 0.1, 0.2, 0, 1, 2)  # only vertical
        if random.random() < 0.5:
            image = flip_axis(image, 1)
            steering_angle = -steering_angle

    return image, steering_angle

generator : train generator
generator_valid : validation generator

In [13]:
def generator(samples, labels, batch_size=128, model='default'):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples, labels)
        
        print('\n training batch shuffled')
        
        for offset in range(0, num_samples, batch_size):
           
        #    print(type(samples))
        #    print(len(samples))
            batch_samples = samples[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            angles = []
            for idx, val in enumerate(batch_samples):
#                name = './IMG/'+batch_sample[0].split('/')[-1]
        
                name = batch_samples[idx]
                
                path = '../merge_image/image/' + name 
            
          #      raw_image = cv2.imread(path)
        
                srcBGR = cv2.imread(path)
          #      destRGB = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
               
                if(model == 'default'):
        
                    new_img = cv2.cvtColor(srcBGR , cv2.COLOR_BGR2YUV)
            
                if(model == 'default_rgb'):
                    new_img = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
               
                raw_angle = float(batch_labels[idx])
                
                input_image, input_angle = process_image_test(new_img, raw_angle)
                
                images.append(input_image)
                angles.append(input_angle)
                
               
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
def generator_valid(samples, labels, batch_size=128, model='default'):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples, labels)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            angles = []
            for idx, val in enumerate(batch_samples):
#                name = './IMG/'+batch_sample[0].split('/')[-1]
        
                name = batch_samples[idx]
                
                path = '../merge_image/image/' + name 
            
                srcBGR = cv2.imread(path)
        #        destRGB = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
        
                if(model == 'default'):
        
                    new_img = cv2.cvtColor(srcBGR , cv2.COLOR_BGR2YUV)
            
                if(model == 'default_rgb'):
                    
                    new_img = cv2.cvtColor(srcBGR, cv2.COLOR_BGR2RGB)
               
                raw_angle = float(batch_labels[idx])
                
                input_image, input_angle = process_image_test(new_img, raw_angle)
                
                images.append(input_image)
                angles.append(input_angle)
                
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

train and valid 
- model_name : model name
  String
- load_stored : whether to restore model or not
  True or False
- load_model_path : file path to save model
  String
- save_best : whether to save model based on Keras, ModelCheckpoint class, monitor parameter value. 
  True or False
- no_epoch : number of epochs


In [14]:
def train_valid(model_name='default', load_stored= False, load_model_path = None, save_best = False, no_epoch=5):
    """Load our network and our data, fit the model, save it."""
    
    # select 'load model' or just 'new model'
    
    print("load_model_path is %s " % load_model_path)
    
   
    
    if(load_stored):
        
 #       net = load_model(load_model_path)
        net = load_model(load_model_path)
        
    else:
            
        net = model(load=False, shape=(160, 320, 3))

    
    # select path to save model based on each model
    
    if(model_name=='default'):
        
        save_path = load_model_path

    if(model_name=='default_rgb'):
        
        save_path = load_model_path

        
    # select data directory 
    data_dir1 = ('../data2/driving_log.csv','linux','left')
 
    # merge data labels from each directory 
    X_merge, y_merge = merge_data1(data_dir1)
#    X_merge, y_merge = merge_data1_2(data_dir7)
    
#    X_merge, y_merge = merge_data3(data_dir1,data_dir2,data_dir3)
    
#    X_merge, y_merge = merge_data4(data_dir1,data_dir2,data_dir3,data_dir4)
#    X_merge, y_merge = merge_data6(data_dir1,data_dir2,data_dir3,data_dir4,data_dir5,data_dir6)

    print(len(X_merge))
    
    
    # shuffle labels
    X_merge,y_merge = shuffle(X_merge, y_merge)
    

    # split data
    train_samples, validation_samples, train_labels, validation_labels = split_data(X_merge,y_merge,split = 0.01)
  
    
    # define generators for training and validation
    # image label should not be dupilcated in the label file for thread safety.
    
    train_generator = generator(train_samples, train_labels, batch_size=128, model=model_name)
    validation_generator = generator_valid(validation_samples, validation_labels, batch_size=128, model=model_name)
    
    
    # define whether to use checkpointer or not
    
    if(save_best):
    
        checkpointer = ModelCheckpoint(filepath=load_model_path, monitor='loss', verbose=1, 
                                   save_best_only=True,save_weights_only=False)
        callbacks_list = [checkpointer]
        
        net.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, 
            nb_val_samples=len(validation_samples), nb_epoch= no_epoch,callbacks=callbacks_list)
        
    else:
        
        net.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, 
            nb_val_samples=len(validation_samples), nb_epoch= no_epoch)
        
        net.save(load_model_path)
        print("Saved model to disk")
       
    
    # model summary for debugging
    net.summary()
    
    # print weights for debugging
    weights_test = net.get_weights()
    
    print(len(weights_test))
    print((np.array(weights_test))[0])

In [15]:
train_valid(model_name='default',load_stored= False, load_model_path = './model/model.h5.30epoch_bump_better', save_best = False, no_epoch=15)

load_model_path is ./model/model.h5.30epoch_bump_better 
768
Epoch 1/15
 training batch shuffled

640/760 [========================>.....] - ETA: 1s - loss: 2.0886 
 training batch shuffled
760/760 [==============================] - 9s - loss: 1.8788 - val_loss: 0.5517
Epoch 2/15
640/760 [========================>.....] - ETA: 1s - loss: 0.8792
 training batch shuffled
760/760 [==============================] - 6s - loss: 0.8235 - val_loss: 1.1452
Epoch 3/15
640/760 [=======

In [16]:
#train_valid(model_name='default_rgb',load_stored= False, load_model_path = './model/model_rgb.h5', save_best = False, no_epoch=15)

In [17]:
"""
def train_valid(model_name='default'):
    """Load our network and our data, fit the model, save it."""
    
    if(model_name=='default'):
        net = model(load=False, shape=(160, 320, 3))
        
        model_save = './model/model.h5'
        
    if(model_name=='nvidia'):
        net = nvidia_model(load=False, shape=(160, 320, 3))
        
        model_save = 'model_nvidia_test.h5'
        
    if(model_name=='nvidia_dropout'):
        net = nvidia_dropout(load=False, shape=(160, 320, 3))
        
        model_save = 'model_nvidia_dropout.h5'
        
#   net = model_nvidia(load=False, shape=(160, 320, 3))
    
    data_dir1 = ('../data2/driving_log.csv','linux')
    
    data_dir2 = ('../20170320/driving_log.csv','window')

    data_dir3 = ('../20170325/driving_log.csv','window')
    data_dir4 = ('../20170325_second/driving_log.csv','window')

   
    data_dir5 = ('../recovery/driving_log.csv','window')
    data_dir6 = ('../recovery2/driving_log.csv','window')
    data_dir7 = ('../recovery3/driving_log.csv','window')
    
    data_dir8 = ('../20170326_1/driving_log.csv', 'window')
    data_dir9 = ('../20170326_2/driving_log.csv', 'window')
    
 #   data_dir10 = ('../20170326_3/driving_log.csv', 'window')
    
    data_dir10 = ('../20170326_4/driving_log.csv', 'window', 'left')
    
    data_dir11 = ('../20170326_5/driving_log.csv', 'window', 'right')
    
    data_dir12 = ('../20170326_6/driving_log.csv', 'window', 'left')
    
    data_dir13 = ('../20170326_7/driving_log.csv', 'window', 'right')
    
    data_dir14 = ('../20170326_8/driving_log.csv', 'window', 'left')
    
    data_dir15 = ('../20170326_9/driving_log.csv', 'window', 'left')
    
    data_dir16 = ('../10270326_10/driving_log.csv', 'window', 'right')
    
    
    X_merge, y_merge = merge_data1(data_dir1)
 #   X_merge, y_merge = merge_data3(data_dir1,  data_dir13, data_dir14)
 #   X_merge, y_merge = merge_data3(data_dir1,  data_dir15, data_dir16)
 #   X_merge, y_merge = merge_data2(data_dir1,  data_dir2)
  #   X_merge, y_merge = merge_data3(data_dir1,  data_dir10, data_dir11)
    
 #   X_merge, y_merge = merge_data3(data_dir1,  data_dir10,  data_dir11)
    
 #   X_merge, y_merge = merge_data4(data_dir1,  data_dir10,   data_dir11, data_dir12)
 #   X_merge, y_merge = merge_data5(data_dir1,  data_dir10,   data_dir11, data_dir15,data_dir16)
    
 #   X_merge, y_merge = merge_data7(data_dir1,  data_dir10,   data_dir11, data_dir12,data_dir13,data_dir15,data_dir16)
    
 #   X_merge, y_merge = merge_data6(data_dir1,  data_dir10,   data_dir11, data_dir12,data_dir13, data_dir15 )
 #   X_merge, y_merge = merge_data(data_dir1,  data_dir10,   data_dir11, data_dir12,data_dir13,data_dir14, data_dir15 )
    
#    X_merge, y_merge = merge_data5(data_dir1)
    
#    X_merge, y_merge = merge_data5(data_dir1, data_dir2, data_dir5,  data_dir6,data_dir7 )
    
#    X_merge, y_merge = merge_data4(data_dir1, data_dir5,  data_dir6,data_dir7 )
    
#    X_merge, y_merge = merge_data(data_dir1, data_dir2, data_dir3,data_dir4,   data_dir5,  data_dir6,data_dir7)
    print(len(X_merge))
    
    X_merge,y_merge = shuffle(X_merge, y_merge)
    
    train_samples, validation_samples, train_labels, validation_labels = split_data(X_merge,y_merge,split = 0.01)
    
    train_generator = generator(train_samples, train_labels, batch_size=128)
    validation_generator = generator_valid(validation_samples, validation_labels, batch_size=128)
    
    checkpointer = ModelCheckpoint(filepath="./model/model.h5", monitor='loss', verbose=1, save_best_only=True)
    callbacks_list = [checkpointer]
        
    net.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, 
            nb_val_samples=len(validation_samples), nb_epoch=30,callbacks=callbacks_list)
    
#    net.save(model_save)
"""

SyntaxError: invalid syntax (<ipython-input-17-f5c7477810a7>, line 3)

In [ ]:
"""
def merge_data(dir1, dir2, dir3,dir4,dir5,dir6 ,dir7):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1])
    
    X4, y4 = get_X_y(dir4[0], flag=dir4[1])
    
    X5, y5 = get_X_y(dir5[0], flag=dir5[1])
    
    X6, y6 = get_X_y(dir6[0], flag=dir6[1])
    
    X7, y7 = get_X_y(dir7[0], flag=dir7[1])
    
    X = X2 + X1 + X3 + X4 + X5 + X6 + X7
    y = y2 + y1 + y3  + y4 + y5 + y6 +y7
    

    return X,y


def merge_data3(dir1, dir2, dir3):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1], turn=dir2[2])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1], turn=dir3[2])
    
   
    
   
    X = X2 + X1 + X3
    y = y2 + y1 + y3 
    

    return X,y


def merge_data4(dir1, dir2, dir3, dir4):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1], turn=dir2[2])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1],turn=dir3[2])
    
    X4, y4 = get_X_y(dir4[0], flag=dir4[1],turn=dir4[2])
    
   
    X = X2 + X1 + X3+ X4
    y = y2 + y1 + y3 +y4  
    

    return X,y


def merge_data5(dir1, dir2, dir3, dir4, dir5):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1],turn=dir2[2])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1],turn=dir3[2])
    
    X4, y4 = get_X_y(dir4[0], flag=dir4[1],turn=dir4[2] )
    
    X5, y5 = get_X_y(dir5[0], flag=dir5[1],turn=dir5[2])
    
   
    X = X2 + X1 + X3+ X4 + X5
    y = y2 + y1 + y3 +y4 +y5 
    

    return X,y

def merge_data6(dir1, dir2, dir3, dir4, dir5, dir6):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1])
    
    X4, y4 = get_X_y(dir4[0], flag=dir4[1])
  
    X5, y5 = get_X_y(dir5[0], flag=dir5[1])
    
    X6, y6 = get_X_y(dir6[0], flag=dir6[1])
    
   
    X = X2 + X1 + X3+ X4 + X5 + X6
    y = y2 + y1 + y3 +y4 +y5 +y6 
    

    return X,y


def merge_data7(dir1, dir2, dir3, dir4, dir5, dir6,dir7):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
    X2, y2 = get_X_y(dir2[0], flag=dir2[1])
    
    X3, y3 = get_X_y(dir3[0], flag=dir3[1])
    
    X4, y4 = get_X_y(dir4[0], flag=dir4[1])
  
    X5, y5 = get_X_y(dir5[0], flag=dir5[1])
    
    X6, y6 = get_X_y(dir6[0], flag=dir6[1])
    
    X7, y7 = get_X_y(dir7[0], flag=dir7[1])
    
   
    X = X2 + X1 + X3+ X4 + X5 + X6 + X7
    y = y2 + y1 + y3 +y4 +y5 +y6  +y7
    

    return X,y


def merge_data1(dir1):

    X1, y1 = get_X_y(dir1[0], flag=dir1[1])
    
   
    X = X1
    y =  y1 
    

    return X,y


"""

In [ ]:
"""
def nvidia_dropout(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

#    conv_layers1, conv_layers2, dense_layers = [24, 36, 48], [64, 64], [1024, 512]
    dense_layers = [1024, 512]
    
    model = Sequential()
    
    model.add(Cropping2D(cropping=((40,20), (0,0)), input_shape=shape))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))

    model.add(Convolution2D(24,5, 5, subsample=(2, 2)))
    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.50))
    
    
    model.add(Convolution2D(36,5, 5, subsample=(2, 2), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.50))
   

    model.add(Convolution2D(48,5, 5, subsample=(2, 2), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.50))


    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), activation='elu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.50))
    
    
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl,W_regularizer=l2(0.001), activation='elu'))
#        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model
"""

In [ ]:
"""
def nvidia_model(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

#    conv_layers1, conv_layers2, dense_layers = [24, 36, 48], [64, 64], [1024, 512]
    dense_layers = [1024, 512]
    
    model = Sequential()
    
    model.add(Cropping2D(cropping=((40,20), (0,0)), input_shape=shape))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))

    model.add(Convolution2D(24,5, 5, subsample=(2, 2), W_regularizer=l2(0.001)))
    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    
    model.add(Convolution2D(36,5, 5, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
   

    model.add(Convolution2D(48,5, 5, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(ELU())
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))


    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), W_regularizer=l2(0.001), activation='elu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu'))
#    model.add(MaxPooling2D(pool_size=(1, 1)))
#    model.add(Dropout(0.50))
    
    
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl,W_regularizer=l2(0.001), activation='elu'))
#        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model
"""

In [ ]:
"""

def get_X_y(data_file, flag='unix', turn='left'):
    """Read the log file and turn it into X/y pairs. Add an offset to left images, remove from right images."""
    X, y = [], []
    right, left, base = [], [], []
    #    images = []
    steering_offset = 0.35
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):

            if(flag == 'window' and turn == 'right'):
 #               print('window right file loaded')
                if (float(speed) > 0 and float(steering_angle) > 0.04):
                    print("steering_angle should be above 0.04 : %s" % steering_angle )
                    right.append(float(steering_angle))

                    for copy in range(2):
                        left_img = left_img.split('\\')[-1]
                        right_img = right_img.split('\\')[-1]
                        center_img = center_img.split('\\')[-1]

                        num_4 = random.random()

                        if (num_4 <= 1 / 3):

                            X += [left_img.strip()]
                            y += [float(steering_angle) + steering_offset]
                        # shuffle(X, y)


                        elif (1 / 3 < num_4 <= 2 / 3):

                            X += [right_img.strip()]
                            y += [float(steering_angle) - steering_offset]

                        else:
                            X += [center_img.strip()]
                            y += [float(steering_angle)]
                            #                        shuffle(X, y)

                        shuffle(X, y)

            elif (flag == 'window' and turn == 'left'):
#                print('window left file loaded')
                if (float(speed) > 0 and float(steering_angle) < -0.04):
                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    left.append(float(steering_angle))

                    for copy in range(5):
                        left_img = left_img.split('\\')[-1]
                        right_img = right_img.split('\\')[-1]
                        center_img = center_img.split('\\')[-1]

                        num_3 = random.random()

                        if (num_3 <= 1 / 3):

                            X += [left_img.strip()]
                            y += [float(steering_angle) + steering_offset]


                        elif (1 / 3 < num_3 <= 2 / 3):

                            X += [right_img.strip()]
                            y += [float(steering_angle) - steering_offset]

                        else:
                            X += [center_img.strip()]
                            y += [float(steering_angle)]
                # shuffle(X, y)

                        shuffle(X, y)

            elif(flag == 'linux'):
 #               print('linux file loaded')

                if (float(steering_angle) > 0.35 or float(steering_angle) < -0.35):
        
                     for copy in range(1):
        
                        left_img = left_img.split('/')[-1]
                        right_img = right_img.split('/')[-1]
                        center_img = center_img.split('/')[-1]
        
                        X += [left_img.strip()]
                        y += [float(steering_angle) + steering_offset]
                
                        base.append(float(steering_angle) + steering_offset)

                        X += [right_img.strip()]
                        y += [float(steering_angle) - steering_offset]
                        
                        base.append(float(steering_angle) - steering_offset)
                       

                        X += [center_img.strip()]
                        y += [float(steering_angle)]
                        
                        base.append(float(steering_angle))

                        shuffle(X, y)

            elif(flag == 'window' and turn == 'recovery'):
                print("window")
    
 #   print("riht anglie sum %s " % sum(right))    
 #   print("left anglie sum %s " % sum(left)) 
    num1 = sum(right)
    num2 = sum(left)
    print("additional data, right turn steering anglie sum :")
    print(sum(right))
    print("additional data, left turn steering anglie sum :")
    print(sum(left))
    print("udacity data anglie sum :")
    print(sum(base))
    return X, y
    
"""

In [ ]:
"""

def get_X_y(data_file, flag='unix', turn='left'):
  
    X, y = [], []
    right, left, base = [], [], []
    #    images = []
    steering_offset = 0.3
    with open(data_file) as fin:
        for center_img, left_img, right_img, steering_angle, _, _, speed in csv.reader(fin):

            if(flag == 'window' and turn == 'right'):
 #               print('window right file loaded')
                if (float(speed) > 0 and float(steering_angle) > 0.04):
                    print("steering_angle should be above 0.04 : %s" % steering_angle )
                    right.append(float(steering_angle))

                    for copy in range(2):
                        left_img = left_img.split('\\')[-1]
                        right_img = right_img.split('\\')[-1]
                        center_img = center_img.split('\\')[-1]

                        num_4 = random.random()

                        if (num_4 <= 1 / 3):

                            X += [left_img.strip()]
                            y += [float(steering_angle) + steering_offset]
                        # shuffle(X, y)


                        elif (1 / 3 < num_4 <= 2 / 3):

                            X += [right_img.strip()]
                            y += [float(steering_angle) - steering_offset]

                        else:
                            X += [center_img.strip()]
                            y += [float(steering_angle)]
                            #                        shuffle(X, y)

                        shuffle(X, y)

            elif (flag == 'window' and turn == 'left'):
#                print('window left file loaded')
                if (float(speed) > 0 and float(steering_angle) < -0.04):
                    print("steering_angle should be below 0.04 : %s" % steering_angle )
                    left.append(float(steering_angle))

                    for copy in range(5):
                        left_img = left_img.split('\\')[-1]
                        right_img = right_img.split('\\')[-1]
                        center_img = center_img.split('\\')[-1]

                        num_3 = random.random()

                        if (num_3 <= 1 / 3):

                            X += [left_img.strip()]
                            y += [float(steering_angle) + steering_offset]


                        elif (1 / 3 < num_3 <= 2 / 3):

                            X += [right_img.strip()]
                            y += [float(steering_angle) - steering_offset]

                        else:
                            X += [center_img.strip()]
                            y += [float(steering_angle)]
                # shuffle(X, y)

                        shuffle(X, y)

            elif(flag == 'linux'):
 #               print('linux file loaded')

                if (float(speed) > 20 and abs(float(steering_angle)) <= 0.04):

                    is_effective = random.random()

                    if (is_effective <= 0.2):
                       

                        for copy in range(3):
                            left_img = left_img.split('/')[-1]
                            right_img = right_img.split('/')[-1]
                            center_img = center_img.split('/')[-1]

                            num_1 = random.random()

                            if (num_1 <= 1 / 3):

                                X += [left_img.strip()]
                                y += [float(steering_angle) + steering_offset]
                                base.append(float(steering_angle) + steering_offset)


                            elif (1 / 3 < num_1 <= 2 / 3):

                                X += [right_img.strip()]
                                y += [float(steering_angle) - steering_offset]
                                base.append(float(steering_angle) - steering_offset)

                            else:
                                X += [center_img.strip()]
                                y += [float(steering_angle)]
                                base.append(float(steering_angle))
                                
                            shuffle(X, y)

                elif(float(speed) > 20 and abs(float(steering_angle)) > 0.04):
                    
                    

                    for copy in range(3):
                        left_img = left_img.split('/')[-1]
                        right_img = right_img.split('/')[-1]
                        center_img = center_img.split('/')[-1]

                        num_2 = random.random()

                        if (num_2 <= 1 / 3):
              #          if (num <= 2 / 7):

                            X += [left_img.strip()]
                            y += [float(steering_angle) + steering_offset]
                            base.append(float(steering_angle) + steering_offset)


                        elif (1 / 3 < num_2 <= 2 / 3):
               #          elif (2 / 7 < num <= 2 / 3):

                            X += [right_img.strip()]
                            y += [float(steering_angle) - steering_offset]
                            base.append(float(steering_angle) - steering_offset)

                        else:
                            X += [center_img.strip()]
                            y += [float(steering_angle)]
                            base.append(float(steering_angle))
                #                        shuffle(X, y)

                        shuffle(X, y)

            elif(flag == 'window' and turn == 'recovery'):
                print("window")
    
 #   print("riht anglie sum %s " % sum(right))    
 #   print("left anglie sum %s " % sum(left)) 
    num1 = sum(right)
    num2 = sum(left)
    print("right anglie sum :")
    print(sum(right))
    print("left anglie sum :")
    print(sum(left))
    print("udacity data anglie sum :")
    print(sum(base))
    return X, y
"""